## Predicción de Fenotipos usando redes neuronales convolucionales e información genómica.


Los datos que urilizaremos serán los del dataset de SoyNAM, el cual recopila información genómica (SNP's) y su correspondiente medición fenotípica, en este caso, la medición de proteína de la soya.

Crearemos una red neuronal convolucional que se encargará de predecir el valor fenotípico mediante una entrada (array) que contiene discretizados aquellos SNP que el especimen en cuestión posee.

La red neuronal al final será evaluada mediante el coeficiente de correlación de Pearson.

El código fue basado en el código que se encuentra en https://github.com/kateyliu/DL_gwas, los comentarios en el código son mi explicación.


Se empieza por importar todas las paqueterías necesarias. Cabe aclarar que se está usando python 2.7, pero debería correr con versiones más actuales.

In [1]:
from __future__ import print_function
import numpy as np
import random
import pandas as pd
from scipy import stats
import sys, os
import logging
import tensorflow as tf
from keras import layers
from keras import regularizers
from keras.models import Model
from keras.models import Sequential
from keras.layers import *
from keras.regularizers import l1,l2, L1L2
from sklearn.metrics.pairwise import cosine_similarity
import keras
import keras.utils.np_utils as kutils
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,Callback,ModelCheckpoint,ReduceLROnPlateau
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import datasets, linear_model
import itertools

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import math as m
import keras.backend as K
import sklearn 

Las funciones definidas a continuación servirán en primer lugar para procesar los datos, pues, necesitamos utilizar la codificación one_hot. Esto es debido a que queremos discretizar cada SNP para después alimentar la red neuronal. Cabe destacar que cada SNP consistirá de un vector de 4 entradas, con un 1 en una entrada y 0 en las demás. Esto representa si el SNP es Homocigoto dominante, Heterocigoto, si no se sabe o si es homocigoto recesivo.

Sobre la red neuronal, es importante notar que el tiempo de ejecución del programa dependerá en gran medida del número de épocas (epoch). En mi caso, usando google colab, tomó 30 minutos realizar el código con 50 épocas.

La red neuronal consta de capas convolucionales, dos capas de dropout para evitar overfitting y una ultima capa densa de neuronas.

In [2]:
nb_classes = 4

def indices_to_one_hot(data,nb_classes):
	
	targets = np.array(data).reshape(-1)
	
	return np.eye(nb_classes)[targets]
	

def readData(input):
	
	data = pd.read_csv(input,sep='\t',header=0,na_values='nan')
	SNP = data.iloc[:,4:].values
	pheno = data.iloc[:,1].values
	folds = data.iloc[:,0].values
	
	arr = np.empty(shape=(SNP.shape[0],SNP.shape[1] , nb_classes))
	
	for i in range(0,SNP.shape[0]):
		arr[i] = indices_to_one_hot(pd.to_numeric(SNP[i],downcast='signed'), nb_classes)
		
	return arr,pheno,folds

	
def resnet(input):
	
	inputs = Input(shape=(input.shape[1],nb_classes))
	
	
	x = Conv1D(10,4,padding='same',activation = 'linear',kernel_initializer = 'TruncatedNormal', kernel_regularizer=regularizers.l2(0.1),bias_regularizer = regularizers.l2(0.01))(inputs)
	
	x = Conv1D(10,20,padding='same',activation = 'linear', kernel_initializer = 'TruncatedNormal',kernel_regularizer=regularizers.l2(0.1),bias_regularizer = regularizers.l2(0.01))(x)
		
	x = Dropout(0.75)(x)
	
	shortcut = Conv1D(10,4,padding='same',activation = 'linear',kernel_initializer = 'TruncatedNormal', kernel_regularizer=regularizers.l2(0.1),bias_regularizer = regularizers.l2(0.01))(inputs)
	x = layers.add([shortcut,x])
	
	x = Conv1D(10,4,padding='same',activation = 'linear',kernel_initializer = 'TruncatedNormal', kernel_regularizer=regularizers.l2(0.1),bias_regularizer = regularizers.l2(0.01))(x)
	
	x = Dropout(0.75)(x)
	x = Flatten()(x)
	
	x = Dropout(0.75)(x)
	
	outputs = Dense(1,activation = isru,bias_regularizer = regularizers.l2(0.01),kernel_initializer = 'TruncatedNormal',name = 'out')(x)
	
	model = Model(inputs = inputs,outputs = outputs)
	model.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adam(lr=0.001),metrics=['mae'])
	
	return model


a= 0.02  

def isru(x):
	return  x/(K.sqrt(1+a*K.square(x)))
	


def model_train(test,val,train,testPheno,valPheno,trainPheno,model_save,weights_save):
	
	batch_size = 250
	earlystop = 5
	epoch = 50
	early_stopping = EarlyStopping(monitor='val_mean_absolute_error', patience=earlystop)
	
	model = resnet(train)
	history = model.fit(train, trainPheno, batch_size=batch_size, epochs=epoch, validation_data=(val,valPheno),callbacks=[early_stopping],shuffle= True)
	
	#model.save(model_save)
	#model.save_weights(weights_save)	
	
	pred = model.predict(test)
	pred.shape = (pred.shape[0],)		
	
	return pred
	

Es momento de procesar los datos para que estos puedan ser asimilados por la red neuronal. Originalmente el artículo usa un método llamado "Ten-Fold Cross-Validation" con el objetivo de disminuir el error estocástico de la predicción, es decir, evitar que los datos quedasen mal distribuidos en la muestra. Pero esto aumenta el tiempo de ejecución 10 veces, por lo tanto decidí utilizar un solo "fold" por ello el rango está entre 1 y 2, es decir i=1 en este código. Pero preservo la oportunidad de aumentar el número de "fold", solo basta cambiar el segundo parámetro del rango a uno más alto (originalmente 11 en vez de 2).

In [3]:
IMP_input =  "IMP_protein.txt"

imp_SNP,imp_pheno, folds = readData(IMP_input)


PHENOTYPE = imp_pheno

for i in range(1,2):
  
  testIdx = np.where(folds == i)
  if i == 10: 
    valIdx = np.where(folds == 1)
    trainIdx = np.intersect1d(np.where(folds != i),np.where(folds != 1))
  else:
    valIdx = np.where(folds == i+1)
    trainIdx = np.intersect1d(np.where(folds != i),np.where(folds != i+1))
  
  trainSNP , trainPheno = imp_SNP[trainIdx], PHENOTYPE[trainIdx]
  valSNP, valPheno = imp_SNP[valIdx],PHENOTYPE[valIdx]
  testSNP,testPheno = imp_SNP[testIdx],PHENOTYPE[testIdx]


A continuación se realiza el entrenamiento de la red neuronal. Guardo el resultado en una variable. Nótese que si se quisiera hacer el "Ten-Fold Cross-Validation" habría que crear un array con las predicciones (o alguna medida de precisión) para cada repetición.

In [4]:
predicciones=model_train(testSNP,valSNP,trainSNP,testPheno,valPheno,trainPheno,'model_IMP/model_'+str(i)+'.txt','model_IMP/model_weights'+str(i)+'.h5')

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/50
17/17 [==============================] - ETA: 0s - loss: 2.0917 - mae: 0.9972

17/17 [==============================] - 41s 2s/step - loss: 2.0917 - mae: 0.9972 - val_loss: 1.4503 - val_mae: 0.8015
Epoch 2/50
17/17 [==============================] - ETA: 0s - loss: 1.5607 - mae: 0.8365

17/17 [==============================] - 39s 2s/step - loss: 1.5607 - mae: 0.8365 - val_loss: 1.3974 - val_mae: 0.7985
Epoch 3/50
17/17 [==============================] - ETA: 0s - loss: 1.4643 - mae: 0.8130

17/17 [==============================] - 40s 2s/step - loss: 1.4643 - mae: 0.8130 - val_loss: 1.3483 - val_mae: 0.7976
Epoch 4/50
17/17 [==============================] - ETA: 0s - loss: 1.4027 - mae: 0.8042

...

17/17 [==============================] - 39s 2s/step - loss: 0.8382 - mae: 0.7151 - val_loss: 0.8554 - val_mae: 0.7262
Epoch 47/50
17/17 [==============================] - ETA: 0s - loss: 0.8388 - mae: 0.7159

17/17 [==============================] - 39s 2s/step - loss: 0.8388 - mae: 0.7159 - val_loss: 0.8537 - val_mae: 0.7270
Epoch 48/50
17/17 [==============================] - ETA: 0s - loss: 0.8299 - mae: 0.7126

17/17 [==============================] - 38s 2s/step - loss: 0.8299 - mae: 0.7126 - val_loss: 0.8529 - val_mae: 0.7272
Epoch 49/50
17/17 [==============================] - ETA: 0s - loss: 0.8247 - mae: 0.7098

17/17 [==============================] - 39s 2s/step - loss: 0.8247 - mae: 0.7098 - val_loss: 0.8525 - val_mae: 0.7254
Epoch 50/50
17/17 [==============================] - ETA: 0s - loss: 0.8283 - mae: 0.7134

17/17 [==============================] - 1s 70ms/step


Vemoas los primeros 10 resultados

In [5]:
predicciones[:10]

array([-0.19971894, -0.25664106, -0.2567696 , -0.35159168, -0.40151888,
       -0.35401353,  0.13065691, -0.40679875, -0.09927557, -0.4699581 ],
      dtype=float32)

Veamos los resultados esperados

In [6]:
testPheno[:10]

array([-0.6124981 ,  0.56616529, -0.2623155 ,  0.74552711,  0.43804971,
        0.75406815, -0.56125186,  0.51491906, -0.7235316 , -0.16836407])

Como podemos observar, es complicado encontrar a simple vista la precisión del modelo. Por lo tanto, en el paper utilizan el coeficiente de correlación de Pearson para medir la precisión de la red neuronal. Hagamos eso:

In [7]:
Coeficiente=pearsonr(predicciones,testPheno)[0]
print(Coeficiente)

0.43603261559434203


De acuerdo con los resultados del paper, este coeficiente de correlación es mejor que el de los demás métodos estadísticos utilizados en la misma tarea. Por lo tanto, aunque aun no tenemos una correlación perfecta, podemos decir que las redes neuronales resultan ser un buen método de predicción de fenotipos utilizando información genómica.